# Extract data from FReD:
## http://reflectance.co.uk


For simplicity, use python selenium to extract the raw spectra and meta data

1> find the HTML table, then use pandas 'read_html' to load into a DataFrame.

2> iterate through the sample IDs and either extract the spectra or save the CSV files. Remember to switch to the 'iframe'

In [172]:
# standard imports
import pandas as pd
import numpy as np
import re
from selenium import webdriver

In [179]:
# start up selenium, put in your own chromedriver directory here:
driver = webdriver.Chrome(r"C:/Users/nebar/OneDrive/Documents/Python Scripts/Web Scrape/chromedriver.exe")
url = "http://reflectance.co.uk//advanceresults.php?bcolourc=Bee%20Colour&hcolourc=Human%20Colour&maincolourc=Main%20Colour&flowersectc=Flower%20Section&altitudec=Altitude&heightc=Height&tubec=tube&corollac=Corolla&pollinatorc=Pollinator&familyc=Family&genusc=Genus&speciesc=Species&countryc=Country&townc=&eastc=&southc=&collectorc=&publicationc=&accessionc=&family=*Any%20Family*&genus=*Any%20Genus*&species=*Any%20Species*&country=*Any%20Country*&town=*Any%20Town*&bcolour=*Any%20Colour*&hcolour=*Any%20Colour*&flowersect=*Any%20Section*&pollinator=*Any%20Pollinator*&collector=*Any%20Collector*&maincolour=*Do%20not%20mind*&altitudegreat=-1&altitudeless=2801&heightgreat=-1&heightless=1001&tubegreat=-1&tubeless=61&corollagreat=-1&corollaless=161"
driver.get(url)

# Extract Meta Data

In [96]:
# keep the string in m
m = driver.find_element_by_xpath("//table").get_attribute("innerHTML")

In [101]:
# parse the string of the html into a DataFrame, then it just happens to be the 3rd [2] dataframe,
# also make sure that the ID is a string, else you lose trailing 0's
raw_df = pd.read_html(m, header=0, converters={"ID": str})[2]

In [102]:
# drop the druplicates and only take samples with ID number that is not NaN
df_clean = raw_df[~raw_df.ID.isnull()].drop_duplicates()

In [103]:
# check it worked and save the raw data
print(f"there are {len(df_clean)} samples")
df_clean.to_pickle("reflectance_meta.pkl")
df_clean.head()

there are 2231 samples


,ID,Bee Colour,Human Colour,Main Colour,Flower Section,Altitude,Height,tube,Corolla,Pollinator,Family,Genus,Species,Country
0,155,blue,blue,NaN,"radially symmetric, whole flower upper side",1060.0,0.0,7.0,5.0,NaN,Boraginaceae,Anchusa,sp.,Italy
2,267,blue,blue,NaN,"radially symmetric, flower tip upper side",1500.0,0.0,0.0,50.0,NaN,Scrophulariaceae,Euphrasia,rostkoviana,Austria
4,431,blue,blue,Y,"zygomorphic, lip lower side",1900.0,0.0,8.0,9.0,NaN,Scrophulariaceae,Euphrasia,rostkoviana,Italy
6,677,blue,pink,Y,flower tip,1200.0,0.0,10.0,14.0,NaN,Fabaceae,Lupinus,polyphyllus,Austria
8,679,blue,white,NaN,keel,1200.0,0.0,10.0,14.0,NaN,Fabaceae,Lupinus,polyphyllus,Austria


# Extract Raw Reflectance

In [ ]:
# get every CSV file
for i in list(set(df_clean.ID)):
    # get the element and click
    m = driver.find_element_by_xpath(f"//table//tr[@id={i}]")
    m.click()
    
    # switch the the 'iframe', the click on download CSV
    driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
    driver.find_element_by_xpath("//*[contains(text(), ' download CSV ')]").click()
    # don't forget to switch back
    driver.switch_to.default_content()


# Close Webdriver

In [ ]:
driver.close()